In [1]:
import pandas as pd
from geopy.distance import vincenty
import time

In [2]:
data_old = pd.read_excel(r"F:/temp/190408/1812JDlnglat.xlsx")
data_old.shape

(39258, 8)

In [3]:
data_new = pd.read_excel(r"F:/temp/190408/1902JDlnglat.xlsx")
data_new.shape

(70243, 8)

In [4]:
data_new.columns

Index(['订单编号', '承运商', 're_province_gaode', 're_city_gaode',
       're_district_gaode', 're_siteLng_gaode', 're_siteLat_gaode',
       're_location_gaode'],
      dtype='object')

In [5]:
df_old = data_old.sample(n=2000)
df_old.shape

(2000, 8)

In [6]:
df_new = data_new.sample(n=2000)
df_new.shape

(2000, 8)

In [7]:
df_old.head()

,订单编号,承运商,re_province_gaode,re_city_gaode,re_district_gaode,re_siteLng_gaode,re_siteLat_gaode,re_location_gaode
19998,690000000001008318122520979404,京东线下,湖北省,宜昌市,西陵区,111.290064,30.708979,"111.290064,30.708979"
32438,690000000001008318120520526356,京东物流,云南省,昆明市,五华区,102.726350,25.098557,"102.726350,25.098557"
14631,690000000001008318121520755129,京东线下,河南省,周口市,商水县,114.610054,33.558082,"114.610054,33.558082"
30623,690000000001008318121920844706,京东物流,浙江省,杭州市,余杭区,120.302055,30.435310,"120.302055,30.435310"
37942,690000000001008318120120411510,京东物流,浙江省,杭州市,余杭区,120.025792,30.357199,"120.025792,30.357199"


In [8]:
df_old = df_old[~df_old["re_district_gaode"].str.contains("\[\]",regex=True)]

df_new = df_new[~df_new["re_district_gaode"].str.contains("\[\]",regex=True)]

In [9]:
print(df_new["承运商"].value_counts())

京东线下    1747
京东物流     213
Name: 承运商, dtype: int64


In [10]:
sheng_list = df_old["re_district_gaode"].unique()
len(sheng_list)

441

In [36]:
def FenSheng(df_new,df_old,col_province,col_city,col_district,col_lng,col_lat):
    start_time = time.time()
    dfA = pd.DataFrame(columns=['orderID_old','orderID_new','distance'])
    limit_KM = round(3 /(3.14 * 6400 /180),6)
    
    
    province_list = df_old[col_province].unique()
    for sheng in province_list:
#         print(sheng)
#         print("*"*30)
        dfold_sheng = df_old[df_old[col_province] == sheng].reset_index(drop=True)
        dfnew_sheng = df_new[df_new[col_province] == sheng].reset_index(drop=True)               
        city_list = dfold_sheng[col_city].unique()
        for shi in city_list:
            print(sheng+shi)
            print("*"*30)
            dfold_shi = dfold_sheng[(dfold_sheng[col_city] == shi)].reset_index(drop=True)
            dfnew_shi = dfnew_sheng[(dfnew_sheng[col_city] == shi)].reset_index(drop=True)               
            district_list = dfold_shi[col_district].unique()  
            print(district_list)
            for xian in district_list:
                dfold = dfold_shi[dfold_shi[col_district] == xian].reset_index(drop=True)
                dfnew = dfnew_shi[dfnew_shi[col_district] == xian].reset_index(drop=True)
                for i in range(0,len(dfnew[col_lng])):
                    for j in range(0,len(dfold[col_lng])):
                        start_lng = round(dfnew[col_lng][i],6)
                        start_lat = round(dfnew[col_lat][i],6)
                        end_lng = round(dfold[col_lng][j],6)
                        end_lat = round(dfold[col_lat][j],6)
                        start_lnglat = (start_lat,start_lng)
                        end_lnglat = (end_lat,end_lng)
                        if (abs(start_lat - end_lat) <= limit_KM or abs(start_lng - end_lng) <= limit_KM ):
                            a = vincenty(start_lnglat, end_lnglat).km
                            row={'orderID_old':dfold['订单编号'][j],'orderID_new':dfnew['订单编号'][i],'distance':a}
                            dfA = dfA.append(row,ignore_index=True)
                        else:
                            pass
                print(str(sheng) + str(shi) + str(xian) + "：已完成")
    end_time = time.time()
    print("总用时：",end_time-start_time)
    return dfA

In [37]:
listA = FenSheng(df_new,df_old,"re_province_gaode","re_city_gaode","re_district_gaode","re_siteLng_gaode","re_siteLat_gaode")

湖北省宜昌市
******************************
['西陵区' '点军区' '伍家岗区' '长阳土家族自治县' '远安县' '枝江市']
湖北省宜昌市西陵区：已完成
湖北省宜昌市点军区：已完成
湖北省宜昌市伍家岗区：已完成
湖北省宜昌市长阳土家族自治县：已完成
湖北省宜昌市远安县：已完成
湖北省宜昌市枝江市：已完成
湖北省武汉市
******************************
['青山区' '汉阳区' '洪山区' '蔡甸区' '武昌区' '硚口区' '江夏区' '新洲区' '东西湖区' '汉南区' '江岸区' '黄陂区'
 '江汉区']
湖北省武汉市青山区：已完成


d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


湖北省武汉市汉阳区：已完成
湖北省武汉市洪山区：已完成
湖北省武汉市蔡甸区：已完成
湖北省武汉市武昌区：已完成
湖北省武汉市硚口区：已完成
湖北省武汉市江夏区：已完成
湖北省武汉市新洲区：已完成
湖北省武汉市东西湖区：已完成
湖北省武汉市汉南区：已完成
湖北省武汉市江岸区：已完成
湖北省武汉市黄陂区：已完成
湖北省武汉市江汉区：已完成
湖北省恩施土家族苗族自治州
******************************
['利川市' '恩施市' '建始县' '鹤峰县' '咸丰县']
湖北省恩施土家族苗族自治州利川市：已完成
湖北省恩施土家族苗族自治州恩施市：已完成
湖北省恩施土家族苗族自治州建始县：已完成
湖北省恩施土家族苗族自治州鹤峰县：已完成
湖北省恩施土家族苗族自治州咸丰县：已完成
湖北省襄阳市
******************************
['襄州区' '老河口市' '襄城区' '保康县' '樊城区' '枣阳市' '南漳县']
湖北省襄阳市襄州区：已完成
湖北省襄阳市老河口市：已完成
湖北省襄阳市襄城区：已完成
湖北省襄阳市保康县：已完成
湖北省襄阳市樊城区：已完成
湖北省襄阳市枣阳市：已完成
湖北省襄阳市南漳县：已完成
湖北省黄冈市
******************************
['团风县' '蕲春县' '罗田县' '浠水县' '黄州区' '麻城市' '红安县' '英山县' '武穴市' '黄梅县']
湖北省黄冈市团风县：已完成
湖北省黄冈市蕲春县：已完成
湖北省黄冈市罗田县：已完成
湖北省黄冈市浠水县：已完成
湖北省黄冈市黄州区：已完成
湖北省黄冈市麻城市：已完成
湖北省黄冈市红安县：已完成
湖北省黄冈市英山县：已完成
湖北省黄冈市武穴市：已完成
湖北省黄冈市黄梅县：已完成
湖北省孝感市
******************************
['孝南区' '应城市' '安陆市' '汉川市' '大悟县']
湖北省孝感市孝南区：已完成
湖北省孝感市应城市：已完成
湖北省孝感市安陆市：已完成
湖北省孝感市汉川市：已完成
湖北省孝感市大悟县：已完成
湖北省鄂州市
******************************
['鄂城区' '华容区']
湖北省鄂州市鄂城区：已完成
湖北省鄂州市华容区：已完成
湖北省十堰市


浙江省杭州市下城区：已完成
浙江省杭州市临安区：已完成
浙江省杭州市淳安县：已完成
浙江省宁波市
******************************
['北仑区' '慈溪市' '余姚市' '镇海区' '鄞州区' '江北区' '海曙区' '宁海县' '奉化区' '象山县']
浙江省宁波市北仑区：已完成
浙江省宁波市慈溪市：已完成
浙江省宁波市余姚市：已完成
浙江省宁波市镇海区：已完成
浙江省宁波市鄞州区：已完成
浙江省宁波市江北区：已完成
浙江省宁波市海曙区：已完成
浙江省宁波市宁海县：已完成
浙江省宁波市奉化区：已完成
浙江省宁波市象山县：已完成
浙江省温州市
******************************
['乐清市' '瑞安市' '龙湾区' '瓯海区' '永嘉县' '文成县' '鹿城区' '苍南县' '平阳县']
浙江省温州市乐清市：已完成
浙江省温州市瑞安市：已完成
浙江省温州市龙湾区：已完成
浙江省温州市瓯海区：已完成
浙江省温州市永嘉县：已完成
浙江省温州市文成县：已完成
浙江省温州市鹿城区：已完成
浙江省温州市苍南县：已完成
浙江省温州市平阳县：已完成
浙江省金华市
******************************
['婺城区' '金东区' '义乌市' '兰溪市' '永康市' '磐安县' '东阳市']
浙江省金华市婺城区：已完成
浙江省金华市金东区：已完成
浙江省金华市义乌市：已完成
浙江省金华市兰溪市：已完成
浙江省金华市永康市：已完成
浙江省金华市磐安县：已完成
浙江省金华市东阳市：已完成
浙江省湖州市
******************************
['南浔区' '安吉县' '德清县' '吴兴区' '长兴县']
浙江省湖州市南浔区：已完成
浙江省湖州市安吉县：已完成
浙江省湖州市德清县：已完成
浙江省湖州市吴兴区：已完成
浙江省湖州市长兴县：已完成
浙江省嘉兴市
******************************
['嘉善县' '桐乡市' '南湖区' '平湖市' '海宁市' '海盐县' '秀洲区']
浙江省嘉兴市嘉善县：已完成
浙江省嘉兴市桐乡市：已完成
浙江省嘉兴市南湖区：已完成
浙江省嘉兴市平湖市：已完成
浙江省嘉兴市海宁市：已完成
浙江省嘉兴市海盐县：已完成
浙江省嘉兴市秀洲区：已完

In [14]:
len(listA)

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_new[df_new["re_province_gaode"] == "江苏省"]

In [ ]:
listA.shape

In [ ]:
listB = listA[listA['distance'] <= 3]

In [ ]:
listB.shape

In [ ]:
# listB.to_excel(r"F:\temp\190408\1812_1902_lnglat_dis.xlsx",index=False)